In [1]:
import pandas as pd

In [2]:
house = pd.read_csv("districts.csv",header=None)
senate = pd.read_csv("senate.csv",header=None)
presidents = pd.read_csv("presidents.csv")


In [3]:
president_poll = pd.read_csv("president_polls.csv")
senate_poll = pd.read_csv("senate_polls.csv")


In [4]:
president_poll_mean = president_poll[president_poll["end_date"] > "10/10/2024"][president_poll["party"] == "REP"].groupby("state")["pct"].mean()

/var/folders/bn/b8mj929d5mq1xcmldttm0_lr0000gn/T/ipykernel_58233/279967254.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  president_poll_mean = president_poll[president_poll["end_date"] > "10/10/2024"][president_poll["party"] == "REP"].groupby("state")["pct"].mean()


In [5]:
senate_poll_mean = senate_poll[senate_poll["end_date"] > "10/10/2024"][senate_poll["party"] == "REP"].groupby("state")["pct"].mean()
senate_poll_mean

/var/folders/bn/b8mj929d5mq1xcmldttm0_lr0000gn/T/ipykernel_58233/1754590078.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  senate_poll_mean = senate_poll[senate_poll["end_date"] > "10/10/2024"][senate_poll["party"] == "REP"].groupby("state")["pct"].mean()


state
Arizona          39.045517
California       35.521429
Connecticut      35.000000
Delaware         31.600000
Florida          47.446170
Indiana          45.750000
Maine            28.000000
Maryland         42.144333
Massachusetts    31.050000
Michigan         40.370526
Minnesota        35.550000
Mississippi      48.000000
Missouri         49.027273
Montana          45.985714
Nebraska         45.500000
Nevada           38.753500
New Jersey       33.600000
New Mexico       37.209091
New York         34.233333
North Dakota     53.500000
Ohio             40.790364
Pennsylvania     41.657652
Rhode Island     32.500000
Tennessee        49.926316
Texas            46.965833
Utah             56.700000
Vermont          25.000000
Virginia         38.161905
Washington       33.555556
West Virginia    45.741176
Wisconsin        44.333171
Name: pct, dtype: float64

In [6]:
presidents["Prediction"] = presidents.value.apply(lambda x: 1- (1 +x) / 2)
presidents = presidents.rename(columns={"state_abb": "State"})[["State","Prediction"]]
presidents

,State,Prediction
0,DC,0.07380
1,VT,0.33165
2,MA,0.35285
3,MD,0.35590
4,HI,0.36380
5,CA,0.37550
6,ME-01,0.38910
7,WA,0.40995
8,CT,0.41185
9,NY,0.41460


In [7]:
presidents.to_csv("Presidential vote share.csv")

In [8]:
import re

# Full state-to-code dictionary as defined above
state_codes = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", 
    "California": "CA", "Colorado": "CO", "Connecticut": "CT", 
    "Delaware": "DE", "Florida": "FL", "Georgia": "GA", "Hawaii": "HI", 
    "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", 
    "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", 
    "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", 
    "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO", 
    "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH", 
    "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", 
    "North Carolina": "NC", "North Dakota": "ND", "Ohio": "OH", 
    "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA", 
    "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD", 
    "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT", 
    "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", 
    "Wisconsin": "WI", "Wyoming": "WY", "District of Columbia": "DC"
}

def format_districts(district):
        formatted = []
        # Extract the state name
        state_match = re.match(r"([A-Za-z\s]+)'s", district)
        if state_match:
            state_name = state_match.group(1).strip()
            state_code = state_codes.get(state_name, "XX")  # Default to 'XX' if not found

            # Extract district number or use 'AL' for At-Large districts
            if "At-Large" in district:
                district_code = "01"
            else:
                district_number = re.search(r"(\d+)", district).group(1)
                district_code = f"{int(district_number):02}"  # Format as 00

            # Combine state code and district code
            formatted.append(f"{state_code}-{district_code}")

        return formatted

In [9]:
senate_final  = pd.DataFrame(columns = ["State", "Prediction"])
count = 0
state = None

for i, v in senate.iterrows():
    if i == 0 or i % 3 == 0:
        state_raw = senate.iloc[i][0] 
        state = state_codes[state_raw] 
    elif (i - 2) % 3 == 0:
            prediction = house.iloc[i][0]
            
            if "R" in prediction:
                diff = float(prediction.split("+")[1])
                percentage = (diff / 100 + 1) / 2
            elif "D" in prediction:
                diff = float(prediction.split("+")[1])
                percentage = 1 - (diff / 100 + 1) / 2
       
            senate_final.loc[count] = [state, percentage]
            count += 1

In [10]:
senate_pred = senate_final.merge(presidents, on="State",how= "left" )
senate_pred["Prediction"] = senate_pred["Prediction_y"] - senate_pred["Prediction_x"]
senate_pred

,State,Prediction_x,Prediction_y,Prediction
0,OH,0.4510,0.54255,0.09155
1,WI,0.3330,0.49725,0.16425
2,PA,0.6930,0.50065,-0.19235
3,MI,0.7095,0.49615,-0.21335
4,TX,1.0000,0.53345,-0.46655
5,FL,1.0000,0.52935,-0.47065
6,AZ,1.0000,0.50865,-0.49135
7,NV,0.5055,0.49900,-0.00650
8,MT,0.5115,0.58720,0.07570
9,NE,0.5140,0.59450,0.08050


In [11]:
senate_pred.to_csv("Senate vote share.csv")

In [12]:
unopposed_r = {
    "AL-03", "AL-04", "AL-05",  "IL-15", "IL-16", 
    "KY-04", "KY-05",  "MS-03", "OK-03", 
    "TX-01",  "TX-11", "TX-13",  "TX-25"
}

unopposed_d = {
    "FL-20", "MA-01", "MA-02", "MA-03", "MA-04", 
    "MA-05", "MA-06", "MA-07", "PA-03", 
    "TX-09", "TX-20"
}


no_major_opposition_d = {
    "CA-37", "TX-30"
}

no_major_opposition_r = {
     "NC-03", "NC-06", "NV-02", "TX-19"
}

same_party_opposition_d = {
    "CA-12", "CA-16", "CA-34",  "WA-09"
}
same_party_opposition_r = {
     "CA-20",  "LA-04", "WA-04"
}

In [13]:
house_final  = pd.DataFrame(columns = ["District", "Prediction"])
count = 0
state = None

for i, v in house.iterrows():
    if i == 0 or i % 3 == 0:
        state_raw = house.iloc[i][0] 
        state = format_districts(state_raw)[0] 
    elif (i - 2) % 3 == 0:
        if state in unopposed_d:
            percentage = 0  
        elif state in unopposed_r:
            percentage = 1.0  
        elif state in no_major_opposition_d:
            percentage = 0
        elif state in same_party_opposition_d:
            percentage = 0
        else:
            prediction = house.iloc[i][0]

            if "R" in prediction:
                diff = float(prediction.split("+")[1])
                percentage = (diff / 100 + 1) / 2
            elif "D" in prediction:
                diff = float(prediction.split("+")[1])
                percentage = 1 - (diff / 100 + 1) / 2
            else:
                percentage = 0.5
        house_final.loc[count] = [state, percentage]
        count += 1
    


   

In [14]:
house_final.sort_values("Prediction")

,District,Prediction
183,MA-07,0.0
377,TX-30,0.0
379,TX-09,0.0
382,TX-20,0.0
180,MA-04,0.0
...,...,...
164,LA-04,1.0
158,KY-05,1.0
157,KY-04,1.0
208,MS-03,1.0


In [15]:
house_final["State"]  = house_final["District"].apply(lambda x : x[:2])
house_pred = house_final.merge(presidents, on="State",how= "left" )
house_pred["Prediction"] = house_pred["Prediction_y"] - house_pred["Prediction_x"]
house_pred

,District,Prediction_x,State,Prediction_y,Prediction
0,AL-02,0.4510,AL,0.63795,0.18695
1,AL-07,0.3330,AL,0.63795,0.30495
2,AL-06,0.6930,AL,0.63795,-0.05505
3,AL-01,0.7095,AL,0.63795,-0.07155
4,AL-03,1.0000,AL,0.63795,-0.36205
...,...,...,...,...,...
430,GA-09,0.6850,GA,0.50740,-0.17760
431,GA-06,0.3050,GA,0.50740,0.20240
432,GA-13,0.3015,GA,0.50740,0.20590
433,GA-04,0.2735,GA,0.50740,0.23390


In [16]:
house_pred.sort_values("Prediction_x").tail(20)

,District,Prediction_x,State,Prediction_y,Prediction
225,NV-02,0.9780,NV,0.49900,-0.47900
376,TX-19,0.9795,TX,0.53345,-0.44605
282,NC-06,0.9815,NC,0.50635,-0.47515
281,NC-03,0.9815,NC,0.50635,-0.47515
303,OK-03,1.0000,OK,0.67120,-0.32880
71,CA-20,1.0000,CA,0.37550,-0.62450
135,IL-16,1.0000,IL,0.42655,-0.57345
134,IL-15,1.0000,IL,0.42655,-0.57345
378,TX-01,1.0000,TX,0.53345,-0.46655
408,WA-04,1.0000,WA,0.40995,-0.59005


In [17]:
house_pred.to_csv("house_pred.csv")